In [17]:
from openai import OpenAI 
import yaml
import os
from dotenv import load_dotenv

load_dotenv()

MODEL="gpt-4o"
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as an env var>")
DB_INDEX_FILE_PATH = os.environ.get("DB_INDEX_FILE_PATH", "<path to the index.json file>")
openAiClient = OpenAI(api_key=OPENAI_API_KEY)

with open("../prompts/memoryClassifier.yml", "r") as file:
    group_classifier = yaml.safe_load(file)

system_prompt = group_classifier["prompt"]

with open(DB_INDEX_FILE_PATH, "r") as file:
    metadata = file.read()

In [18]:
def create_messages(input, metadata):
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": f"Query: ${input}"
        },
        {
            "role": "user",
            "content": f"Metadata: ${metadata}"
        }        
    ]

In [23]:
# input = "notes about KP-API"
input = "any memory about vacation"
completion = openAiClient.chat.completions.create(
    model=MODEL,
    temperature=0,
    stream=False,
    response_format={"type": "json_object"},
    messages=create_messages(input, metadata)),

print(completion[0].choices[0].message.content)

{
  "thinking_process": "The user's query is about memories related to vacations. Based on the metadata provided, the relevant category for this query is 'Areas' as it includes long-term responsibilities and interests, which can encompass travel and vacation plans.",
  "keys": [
    "AREAS/TRAVELLING/galicia",
    "AREAS/TRAVELLING/roadmap_23-24",
    "AREAS/TRAVELLING/travelling"
  ]
}
